# Arbeitspaket (AP) 2: Retrieval Augmented Generation (RAG)

**Datenthema:** Jugend, Politik, Umwelt & Partizipation
  
**Maximale Punktzahl:** 60 Punkte

---
## Hinweise
- Beantworten Sie alle Aufgaben direkt in diesem Notebook.
- Ändern Sie Zellen mit dem Hinweis **"Bitte nicht verändern"** nicht.
- Verwenden Sie, wo möglich, die bereits gegebenen Variablen und Funktionen.
- Antworten auf Theoriefragen schreiben Sie in Markdown-Zellen.

Die Daten für den RAG-Teil (Texte zu *Jugend, Politik, Umwelt & Partizipation* in deutscher Sprache) müsssen im Ordner `data/` abgelegt werden. Sie benötigen ebenfalls eine `.env`-Datei für die Keys.

---
## Teil A – LLM API-Aufrufe (25 Punkte)


### Persönliche Angaben (bitte ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td>Colin</td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td>Zemp</td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td>17679390</td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>15.12.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.24HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

| **Kategorie**                       | **Beschreibung**                                                                                                                                          | **Punkteverteilung**                |
|-------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------|
| **Code nicht lauffähig oder Ergebnisse nicht relevant** | Der Code läuft nicht oder erfüllt nicht die Anforderungen des Aspekts (z. B. Bilder werden nicht geladen, die Textausgabe der Extraktion fehlt, Bounding Boxes werden nicht angezeigt). | **0 Punkte**                       |
| **Code lauffähig, aber mit gravierenden Mängeln**       | Der Code läuft, jedoch fehlen zentrale Teile der Funktionalität eines Aspekts (z. B. unvollständige Extraktion von Bildinformationen oder Fehler bei der Definition eines Schemas). | **25% der max. erreichbaren Punkte** |
| **Code lauffähig, aber mit mittleren Mängeln**          | Der Code läuft und liefert teilweise korrekte Ergebnisse für einen Aspekt, aber wichtige Details fehlen (z. B. ungenaue Bounding Boxes, unvollständige Integration der extrahierten Daten). | **50% der max. erreichbaren Punkte** |
| **Code lauffähig, aber mit minimalen Mängeln**          | Der Code erfüllt die Anforderungen eines Aspekts weitgehend, aber kleinere Fehler oder Abweichungen (z. B. nicht robust Extraktionsdaten, kleinere Schemaabweichungen, Prompt zu wenig stringent formuliert -> teilweise unstabile Output)sind vorhanden. | **75% der max. erreichbaren Punkte** |
| **Code lauffähig und korrekt**                         | Der Code erfüllt die Anforderungen des Aspekts vollständig und liefert die erwarteten Ergebnisse ohne Fehler (z. B. korrekte Extraktion, vollständige Bounding Boxes, saubere Integration). | **100% der max. erreichbaren Punkte** |

---


### A0. Setup (0 Punkte)

Führen Sie diese Zellen aus, nachdem Sie die `.env` Datei mit Ihren API-Keys in den Codespace gezogen haben.  

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# === A0. Setup (Bitte Schlüssel nur lokal eintragen, nicht ins Versionskontrollsystem hochladen) ===
openai_api_key = os.getenv("OPENAI_API_KEY")
deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")

# OpenAI-Client
openai_client = OpenAI(api_key=openai_api_key)

# Deepinfra-Client (OpenAI-kompatible API)
deepinfra_client = OpenAI(
    api_key=deepinfra_api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

print("Setup abgeschlossen.")

Setup abgeschlossen.


### A1. Funktion `ask_openai` (10 Punkte)

Implementieren Sie eine Funktion `ask_openai`, die eine Anfrage mit Kontext an das OpenAI-Chat-API schickt und die Antwort als String zurückgibt.

**Anforderungen:**
1. Signatur:
   ```python
   def ask_openai(user_input: str, context: str) -> str:
       ...
   ```
2. Verwenden Sie den bereits definierten `openai_client` aus A0.
3. Nutzen Sie das Modell `"gpt5-mini"`.
4. Die Funktion soll **nur den Textinhalt** der Antwort zurückgeben (ohne zusätzliche Prints).
5. Der Kontext soll in den Systemprompt (injected) werden.

Testen Sie Ihre Funktion danach mit **zwei Aufrufen**:
- einmal mit dem Prompt: *"Schreibe ein Haiku über Rekursion in der Programmierung."*
- einmal mit dem Prompt: *"Erkläre in einfachen Worten, was der Parameter `temperature` bei LLMs macht."*

Drucken Sie beide Antworten aus.

In [10]:
user_input1 = "Schreibe ein Haiku über Rekursion in der Programmierung."
context1 = """
Rekursion ist eine Technik in der Programmierung, bei der eine Funktion sich selbst aufruft, um ein Problem in kleinere Teilprobleme zu zerlegen. 
Typischerweise gibt es dabei eine sogenannte Basisbedingung, bei der die Funktion ohne weiteren Selbstaufruf endet. 
Dadurch lassen sich komplexe Strukturen wie Bäume oder verschachtelte Listen oft sehr elegant bearbeiten. 
Allerdings kann Rekursion auch zu Fehlern führen, wenn die Basisbedingung fehlt oder nie erreicht wird. 
In vielen Fällen gibt es sowohl eine rekursive als auch eine iterative Lösung, die sich in Verständlichkeit und Effizienz unterscheiden können.
"""

user_input2 = "Erkläre in einfachen Worten, was der Parameter `temperature` bei LLMs macht."
context2 = """
Der Parameter 'temperature' steuert bei vielen Sprachmodellen, wie kreativ oder deterministisch die Antworten sind. 
Bei einer niedrigen Temperatur (z.B. 0.1) wählt das Modell eher die wahrscheinlichsten Worte und verhält sich dadurch konservativ und vorhersehbar. 
Bei einer höheren Temperatur (z.B. 0.8 oder 1.0) werden auch unwahrscheinlichere Worte stärker berücksichtigt, was zu vielfältigeren und kreativeren Antworten führt. 
Zu hohe Werte können allerdings dazu führen, dass die Ausgabe unzusammenhängend oder unsinnig wird. 
Die Wahl einer passenden Temperatur hängt daher vom Anwendungsfall ab, etwa ob man eher präzise Fakten oder kreative Ideen haben möchte.
"""

In [7]:
def ask_openai(user_input: str, context: str) -> str:
    completion = openai_client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": context},
            {
                "role": "user",
                "content": user_input
            }
        ]
    )

    return completion.choices[0].message.content

In [8]:
# the two executions
print(ask_openai(user_input1, context1)+"\n\n")
print(ask_openai(user_input2, context2))

Die Funktion ruft sich
Zerlegt das Problem weiter
Bis die Basis ruft


Kurz und einfach: Die Temperatur steuert, wie „zufällig“ oder „sicher“ ein Sprachmodell bei der Wortwahl ist.

- Niedrige Temperatur (z.B. 0.0–0.3): das Modell wählt meist die wahrscheinlichsten Wörter — die Antworten sind vorhersehbar, präzise und konservativ.  
- Mittlere Temperatur (z.B. 0.4–0.7): ein Kompromiss aus Verlässlichkeit und etwas Vielfalt.  
- Hohe Temperatur (z.B. 0.8–1.0+): das Modell probiert auch weniger wahrscheinliche Wörter — die Ausgaben werden kreativer, aber auch unvorhersehbarer oder fehleranfälliger.

Analog: Bei niedriger Temperatur wählt das Modell immer das „sicherste“ Wort, bei hoher Temperatur zieht es öfter ungewöhnliche Ideen aus dem Hut. Für Faktenantworten niedrig wählen, für kreative Texte höher.


### A2. Deepinfra-Aufruf korrigieren und erweitern (10 Punkte)

Unten finden Sie einen (bewusst) fehlerhaften Beispielaufruf an ein Modell über die Deepinfra-API.  
**Ihre Aufgaben:**

1. **Korrigieren** Sie den Aufruf so, dass er syntaktisch korrekt ist und mit dem `deepinfra_client` funktioniert.  
   - Achten Sie insbesondere auf Parameter-Namen und den Zugriff auf das Antwortobjekt.
2. Erweitern Sie den Code so, dass **zwei Gesprächsrunden** mit dem Modell stattfinden:
   - Erste User-Frage: *"Nenne zwei unterschiedliche Möglichkeiten, wie Jugendliche sich für den Umweltschutz politisch engagieren können."*
   - Zweite User-Frage: *"Erläutere die zweite Möglichkeit etwas genauer."*
   Verwenden Sie dafür die gleiche `messages`-Liste und fügen Sie nur die neue Nutzer-Nachricht und die Assistenten-Antwort hinzu.
3. Implementieren Sie eine einfache **Längenbegrenzung**:  
   Wenn die endgültige Antwort-Variable mehr als **400 Zeichen** enthält, kürzen Sie sie auf 400 Zeichen und hängen Sie `"..."` an.

Verwenden Sie ein geeignetes deutschsprachiges oder multilinguales Modell aus Deepinfra (z.B. ein Llama- oder Mixtral-Chatmodell).

In [5]:
# A2. Deepinfra-Aufruf – fehlerhafte Ausgangsversion
# TODO (A2): Korrigieren und erweitern Sie diesen Code.

model_name = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"  # Beispiel – ggf. anpassen

messages = [
    {"role": "system", "content": "Du bist eine sachliche, gut informierte Assistenz."},
    {"role": "user", "content": "Nenne zwei unterschiedliche Möglichkeiten, wie Jugendliche sich für den Umweltschutz politisch engagieren können."}
]

chat_completion = deepinfra_client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8
)

answer1 = {"role": "assistant", "content": chat_completion.choices[0].message.content}
messages.append(answer1)
question2 = {"role": "user", "content": "Erläutere die zweite Möglichkeit etwas genauer."}
messages.append(question2)

chat_completion2 = deepinfra_client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8
)
answer2 = chat_completion2.choices[0].message.content
print(answer2+"\n\n")

a2_short = (answer2[:400] + '...') if len(answer2) > 400 else answer2
print(a2_short)
pass

Die zweite Möglichkeit, sich für den Umweltschutz politisch zu engagieren, besteht darin, dass Jugendliche sich in politischen Parteien oder Initiativen engagieren, die sich für den Umweltschutz einsetzen.

**Politische Parteien:**

*   Jugendliche können Mitglieder von Parteien wie Bündnis 90/Die Grünen werden, die sich stark für den Umweltschutz einsetzen.
*   Sie können an Parteitagen teilnehmen, sich in Arbeitsgruppen engagieren und ihre Meinung zu umweltpolitischen Themen äußern.
*   Durch die Mitarbeit in einer Partei können Jugendliche Einfluss auf die politische Agenda nehmen und sich für umweltfreundliche Politiken einsetzen.

**Initiativen:**

*   Jugendliche können sich auch in lokalen Initiativen wie Klimagruppen, Umweltverbänden oder Bürgerinitiativen engagieren.
*   Diese Initiativen setzen sich oft für spezifische Umweltziele ein, wie z.B. die Reduzierung von Plastikmüll oder die Förderung erneuerbarer Energien.
*   Durch die Mitarbeit in einer Initiative können Jugendli

### A3. Kurzfrage: Unterschiedliche Provider (5 Punkte)

Beantworten Sie die folgende Frage in **3–4 Sätzen** in der Markdown-Zelle darunter:

> Nennen Sie **einen Vorteil** der Nutzung eines gehosteten Dienstes wie OpenAI im Vergleich zu einem selbst gehosteten Modell,
> und nennen Sie **mindestens einen Nachteil bzw. ein Risiko**.

Gehen Sie dabei kurz auf Aspekte wie zum Beispiel Leistung, Kosten, Flexibilität oder Datenschutz ein.

**Antwort A3:**
Gehostete Dienste erlauben den Zugriff auf grundsätzlich "bessere" Modelle (Foundation Models wie GPT5.1), die mit sehr teurer Hardware trainiert wurden und auf teurer Hardware laufen. Diese sind lokal gar nicht oder nur begrenzt (z.B. GPT OSS) verfügbar und selbst wenn, wäre die benötigte Hardware dafür sehr teuer. Sie sind ausserdem meist stabiler verfügbar (bei den Hostern gibt es Ausfallsicherheit, SLAs, ...)

Grösstes Risiko / Problem davon ist Datenschutz. Es kann nicht begrenzt werden wohin die Daten fliessen und maximal vertraglich festgelegt werden, wer Zugriff auf diese Daten hat bzw. wie diese verarbeitet oder zum Lernen der Modelle weiterverwendet werden. Ausserdem ist die Gefahr für Lock-In Risiken gross und bei closed-source / closed-parameter Modellen weiss man zudem nicht, wie sie gelernt haben.

---
## Teil B – Retrieval-Augmented Generation (RAG) (35 Punkte)

In diesem Teil arbeiten Sie mit einer kleinen RAG-Pipeline.  
Die zugrundeliegenden Texte behandeln das Thema **"Jugend, Politik, Umwelt & Partizipation"** in deutscher Sprache.

Die folgenden Objekte werden in vorbereiteten Zellen erstellt:
- `documents`: Liste von Rohtexten (z.B. aus PDF- oder Textdateien)
- `chunks`: Liste von Textchunks
- `embedder`: Objekt mit einer Methode `.encode(texts, convert_to_numpy=True)`
- `index`: ein FAISS-Index, der Embeddings von `chunks` enthält

Sie sollen nun die fehlenden Teile der Pipeline implementieren bzw. anpassen.


### B0. RAG-Setup (0 Punkte – bitte nicht verändern)

Diese Zelle lädt die Dokumente, erzeugt Chunks, berechnet Embeddings und baut den FAISS-Index auf.  
**Verändern Sie diese Zelle nicht.**

In [ ]:
# === B0. RAG-Setup (Bitte nicht verändern) ===
import os
from typing import List
import glob
from PyPDF2 import PdfReader
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pickle
from sentence_transformers import SentenceTransformer
import faiss

# Dokumente Laden

DATA_DIR = "data/*.pdf"  # Ordner mit Texten zu "Jugend, Politik, Umwelt & Partizipation"

text = ""

for pdf_path in glob.glob(DATA_DIR):
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += " " + page_text

# Split into chunks

splitter_a = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=250
)

chunks = splitter_a.split_text(text)

print(f"Anzahl Chunks: {len(chunks)}")

# 3. Embedder und Embeddings
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True)

# 4. FAISS-Index erstellen
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("RAG-Setup abgeschlossen.")


os.makedirs("faiss", exist_ok=True)
faiss.write_index(index, "faiss/faiss_exam.index")
with open("faiss/chunks_exam.pkl", "wb") as f:
    pickle.dump(chunks, f)

### Run in case of crash later on

In [ ]:
# index = faiss.read_index("faiss/faiss_exam.index")

# with open("faiss/chunks_exam.pkl", "rb") as f:
#    chunks = pickle.load(f)

### B1. Funktion `retrieve_texts` korrigieren (10 Punkte)

Die folgende Funktion soll zu einer Anfrage (`query`) die **Top-`k` ähnlichsten Chunks** zurückgeben.
Die Implementierung ist fehlerhaft.

**Ihre Aufgaben:**
1. Korrigieren Sie die Funktion so, dass sie:
   - die Query korrekt mit dem `embedder` embedden kann,
   - den FAISS-Index `index` korrekt abfragt und
   - eine **Liste von Strings** mit den gefundenen Chunks zurückgibt.
2. Achten Sie insbesondere auf die richtige Form der Embedding-Arrays und den Umgang mit den Indizes.

Verwenden Sie die bereits existierenden Objekte `index`, `chunks` und `embedder`.

In [ ]:
from typing import List

def retrieve_texts(query: str, k: int, index, chunks: List[str], embedder) -> List[str]:
    """Gibt die Top-k ähnlichsten Chunks für eine Query zurück.

    TODO (B1): Funktion korrigieren.
    """
    # FEHLERHAFTE BEISPIELIMPLEMENTIERUNG:
    query_emb = embedder.encode(query, convert_to_numpy=True)  # Form kann problematisch sein
    distances, indices = index.search(query_emb, k)  # hier kann ein Fehler auftreten
    return [chunks[indices]]  # ebenfalls fehlerhaft

# Sie können die Funktion z.B. mit einer Test-Query ausprobieren:
test_query = "Wie können Jugendliche sich politisch für den Klimaschutz engagieren?"
try:
    test_results = retrieve_texts(test_query, k=3, index=index, chunks=chunks, embedder=embedder)
    for i, r in enumerate(test_results, start=1):
        print(f"Chunk {i} (Ausschnitt):", r[:200], "\n---\n")
except Exception as e:
    print("Fehler bei Testabfrage (erwartet, solange die Funktion noch nicht korrigiert ist):", e)

### B2. RAG-Antwortfunktion `answer_query` anpassen (10 Punkte)

Unten sehen Sie eine Funktion `answer_query`, die eine Nutzerfrage beantwortet, indem sie:
1. mit `retrieve_texts` passende Chunks zum Thema findet,
2. diese als Kontext an das LLM schickt und
3. eine Antwort generiert.

Die Funktion ist noch nicht korrekt auf das Thema **"Jugend, Politik, Umwelt & Partizipation"** und auf RAG zugeschnitten.

**Ihre Aufgaben:**
1. Passen Sie den `system_prompt` an die neue Domäne an und geben sie klare Anweisungen wie mit dem Kontext umzugehen ist.  
3. Verwenden Sie das Modell `"gpt-5-mini"` über den `openai_client` aus Teil A.
4. Wählen Sie einen sinnvollen Standardwert für `k`und kommentieren Sie in **einem kurzen Kommentar im Code**, warum diese Wahl für diesen Anwendungsfall plausibel ist.


In [ ]:
def answer_query(query: str, k: int, index, chunks: List[str], embedder, client: OpenAI) -> str:
    """Beantwortet eine Nutzerfrage mittels RAG.

    TODO (B2):
    - Prompt anpassen
    - Sicherstellen, dass nur Kontext verwendet wird
    - openai_client und gpt-5-mini nutzen
    """
    retrieved_chunks = retrieve_texts(query, k, index, chunks, embedder)
    context = "\n\n---\n\n".join(retrieved_chunks)

    system_prompt = (
        "Du bist eine hilfreiche Assistenz, die Fragen zu MEDIZINISCHEN LEITLINIEN beantwortet. "
        "(TODO: Dies ist noch nicht an die neue Domäne angepasst.)"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Kontext:\n{context}\n\nFrage: {query}"},
    ]

    completion = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
    )

    return completion.choices[0].message.content.strip()

# Beispielaufruf (wenn B1 und B2 korrekt implementiert sind):
example_query = "Welche Formen politischer Beteiligung von Jugendlichen im Umweltschutz werden im Text genannt?"
try:
    answer = answer_query(example_query, k=, index=index, chunks=chunks, embedder=embedder, client=openai_client)
    print(answer)
except Exception as e:
    print("Fehler bei Beispielaufruf (erwartet, solange B1/B2 noch nicht fertig sind):", e)

### B3. Einfache Retrieval-Sanity-Checks (5 Punkte)

Erweitern Sie die Funktion `answer_query` aus B2 wie folgt:

1. **Vor dem LLM-Aufruf** soll ein kurzer Ausschnitt (ca. die ersten 200 Zeichen) des **besten (Top-1) Chunks** ausgegeben werden, z.B. mit `print("Top-1 Chunk:", ...)`.
2. Falls aus irgendeinem Grund **keine Chunks** zurückgegeben werden (z.B. leere Liste), soll die Funktion **keinen LLM-Aufruf** machen, sondern einen verständlichen Hinweis-String zurückgeben, z.B.:
   > "Es konnten keine passenden Textstellen gefunden werden."

Implementieren Sie diese Änderungen direkt in der Funktion `answer_query` oben.

_Hinweis: Sie müssen hier keine neue Zelle schreiben. Passen Sie die bestehende Implementierung von `answer_query` direkt an._

### B4. Fragen und Kriterien für eine Evaluation des RAG-Systems (10 Punkte)

Sie haben in diesem Notebook ein RAG-System aufgebaut, das Fragen zum Thema
**„Jugend, Politik, Umwelt & Partizipation“** beantwortet.

In dieser Aufgabe sollen Sie das LLM nutzen, um:

1. sinnvolle **Evaluationsfragen** zu generieren, mit denen man das System testen könnte, und
2. **Kriterien** vorschlagen zu lassen, auf die man bei der Evaluation achten sollte.

---

#### B4.1 – Funktion `generate_eval_questions` implementieren

Implementieren Sie eine Funktion, die mit Hilfe des OpenAI-Clients _**10**_ Evaluationsfragen generiert.

**Anforderungen:**

1. Signatur:
   ```python
   def generate_eval_questions(chunks, num_chunks: int = 5) -> str:
       ...

In [ ]:
def generate_eval_questions(chunks, num_chunks) -> list:

    # B4.2 – Kriterien für die Auswertung generieren

    # TODO (B4.2):
    # - eigenen Systemprompt definieren (Evaluationsexpert*in) 
    # - jeden chunk aus dem sample in den prompt injecten
    # - jede generierte Frage in die Liste einfügen

    questions = []
    prompt_eval = (
        "TODO: Formulieren Sie hier eine Anfrage, in der Sie das Modell bitten, "
        "Fragen zu genierieren, die zur Evaluation eines "
        "RAG-Systems (Jugend, Politik, Umwelt & Partizipation) achten sollte. " 
        "Es soll genau eine Frage, und nur die Frage ausgegeben werden"
    )

In [ ]:
questions = generate_eval_questions()
print(questions)

---
## Ende der Prüfung

Überprüfen Sie, ob alle Code-Zellen ausgeführt wurden und alle Antworten (auch die theoretischen Fragen) eingetragen sind.

Viel Erfolg!

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')